In [1]:
import sys
sys.path.insert(0,'/home/misa/git_repositories/APDFT/prototyping/atomic_energies')
from parse_pyscf import read_input
import qml
import numpy as np
import os
import utils_qm as uqm
from ase import units
import pyscf.qmmm

In [2]:
def make_apdft_calc(deltaZ, includeonly, mol, method, **kwargs):
    """
    SCF calculation for fractional charges defined in deltaZ
    returns the density matrix and the total energy
    """
    
    if 'verbose' in kwargs.keys():
        verbose = kwargs['verbose']
    else:
        verbose = 0
    
    
    if method == "HF":
        calc = add_qmmm(pyscf.scf.RHF(mol), deltaZ, includeonly, mol)
        for k in kwargs.keys():
            if k == 'max_cycle':
                calc.max_cycle = kwargs[k]
            elif k == 'diis':
                calc.diis = kwargs[k]
            elif k == 'init_guess':
                calc.init_guess = kwargs[k]

        
    elif method == "DFT":
        calc = add_qmmm(pyscf.dft.RKS(mol), deltaZ, includeonly, mol)
        calc.xc = 'lda,vwn'
        for k in kwargs.keys():
            if k == 'max_cycle':
                calc.max_cycle = kwargs[k]
            elif k == 'diis':
                calc.diis = kwargs[k]
            elif k == 'init_guess':
                calc.init_guess = kwargs[k]
    hfe = calc.kernel(verbose=verbose)
    dm1_ao = calc.make_rdm1()
    total_energy = calc.e_tot

    return(dm1_ao, total_energy, calc.mo_energy, calc.mo_occ)

def add_qmmm(calc, deltaZ, includeonly, mol):
    """
    modify hamiltonian such that Z = Z(lambda)
    """
    mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]*units.Bohr, deltaZ) # add charge dZ at position of nuclei

    def energy_nuc(self):
        """
        calculate correct nuclear charge for modified system
        """
        q = mol.atom_charges().astype(np.float)
        q[includeonly] += deltaZ
        return mol.energy_nuc(q)

    mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

    return mf

In [3]:
# get current directory
run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/qm9_data/dsgdb9nsd_003664'
#run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/qm9_data/dsgdb9nsd_001711'  #os.getcwd()
os.chdir(run_dir)
print('Initializing')
input_parameters = read_input(os.path.join(run_dir, 'input_parameters'))
inputfile = input_parameters['structure_file']#os.path.join(basepath, com+'.xyz')
intg_meth = input_parameters['intg_meth']
basis = 'augccpvtz'#input_parameters['basis'] # 'def2-qzvp'
com = qml.Compound(xyz=inputfile)

#lam_vals = np.concatenate((np.zeros(1), lam_vals))
alchemical_potentials = []
#alchemical_potentials.append(np.zeros(len(com.nuclear_charges)).tolist())
mo_energies = []
mo_occs = []
dm_restart = None

Initializing


In [4]:
mol = pyscf.gto.Mole()
for ch, coords_atom in zip(com.nuclear_charges, com.coordinates):
    mol.atom.append([ch, coords_atom])
mol.basis = basis
mol.build()
# dZ vector to generate systems for lambda != 1
deltaZ = com.nuclear_charges*0.95-com.nuclear_charges
includeonly = np.arange(len(mol.atom_coords()))

In [5]:
dm1_ao, total_energy, mo_energy, mo_occ = make_apdft_calc(deltaZ, includeonly, mol, "DFT", **{'init_guess':'atom'})

Overwritten attributes  energy_nuc  of <class 'pyscf.qmmm.itrf.qmmm_for_scf.<locals>.QMMM'>


SCF not converged.
SCF energy = -299.01516389061


In [ ]:
len(np.where(mo_energy < 0)[0])*2

In [ ]:
mo_occ.sum()

In [ ]:
dm1_ao.shape